In [ ]:
# Import libaries
import pandas as pd
import numpy as np
import requests
import os
import pprint

# Set option to view max 500 columns
pd.set_option('display.max_columns', 500)

In [ ]:
# Recall the bike stations latitude and logitude from our saved 'montreal_stations_coordinate.csv' file
montreal_stations_coordinate = pd.read_csv('../data/montreal_stations_coordinate.csv')
montreal_stations_coordinate

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# As Yelp API has a daily limit of 500 per day, we'll take split into two sets of the coordinates at 400 rows for Yelp
montreal_stations_coordinate1 = montreal_stations_coordinate[:400]
montreal_stations_coordinate2 = montreal_stations_coordinate[400:]

In [ ]:
# The new Montreal station coordinates dataframes to iterate and make Yelp API call is now 'montreal_stations_coordinate1' and 'montreal_stations_coordinate2'
# Scheduled to run 2 sub-groups in two different days to bypass the Yelp API limit
print("Length of the first set of montreal bike stations coordinates :", len(montreal_stations_coordinate1))
print("Length of the first set of montreal bike stations coordinates :", len(montreal_stations_coordinate2))

In [ ]:
YELP_KEY1 = os.getenv('yelp_key1')

In [ ]:
# Due to the 5000 limit per day, let's test out a small sample of 5 coordinates to figure out the response data structure from Yelp API
montreal_stations_coordinate_s = montreal_stations_coordinate[:5]

# Create a test function to get sample businesses from Yelp API with limit 
def get_yelp_business_sample(latitude, longitude, radius, api_key, categories, limit):

    # Form the url with a limit per coordinate
    yelp_url_limit = 'https://api.yelp.com/v3/businesses/search?' + f"latitude={latitude}&longitude={longitude}" + f"&radius={radius}" + f"&categories={categories}" + f"&limit={limit}"
    
    # Create dictionary for headers, add key with our API KEY
    headers = {"Accept": "application/json",
              "Authorization": f"Bearer {api_key}"}
    
    # Get the response
    yelp_result_sample = requests.get(yelp_url_limit, headers=headers)
    
    return yelp_result_sample.json()

In [ ]:
# Create an empty DataFrame to store the sample results from Yelp for bars with 1000m radius from Montreal bike stations, limit=1 for each coordinate
yelp_bars_df_s = pd.DataFrame()

for index, row in montreal_stations_coordinate_s.iterrows():
    latitude = row['cb_latitude']
    longitude = row['cb_longitude']
    yelp_bar_data_sample = get_yelp_business_sample(latitude=latitude, longitude=longitude, radius=1000, api_key=YELP_KEY1, categories="bars", limit=1)

    # Extract the 'businesses' key from the JSON response and create a DataFrame from it
    yelp_businesses_sample = yelp_bar_data_sample.get('businesses', [])
    yelp_bar_details_sample = pd.DataFrame(yelp_businesses_sample)

    # Add latitude and longitude to yelp_bar_details_sample DataFrame
    yelp_bar_details_sample['yelp_latitude'] = latitude
    yelp_bar_details_sample['yelp_longitude'] = longitude

    # Append it to 'yelp_bars_df_s'
    yelp_bars_df_s = pd.concat([yelp_bars_df_s, yelp_bar_details_sample], ignore_index=True)

In [ ]:
yelp_bars_df_s.head()

In [ ]:
# The five sample coordinate API call works with the 'businesses' key
# Let's apply it to the two sets of the coordinates at 400 row with an official function

# Create a function to get businesses from Yelp API
def get_yelp_business(latitude, longitude, radius, api_key, categories):

    """
    Get business information from Yelp API based on latitude, longitude, radius, and categories.

    Parameters:
        latitude (float): The latitude coordinate of the location.
        longitude (float): The longitude coordinate of the location.
        radius (int): The search radius in meters.
        api_key (str): Your Yelp API key for authentication.
        categories (str): A comma-separated list of venue categories.
        
    Returns:
        json: JSON data containing the response data from the Yelp API.

    Example:
        response_data = get_yelp_business(latitude=40.7128, longitude=-74.0060, radius=1000, "YOUR_API_KEY", categories="bars")
        # Returns JSON data with business information based on the provided parameters.
    """

    # Form the url with a limit as Yelp API has a daily limit of 500 per day
    yelp_url = 'https://api.yelp.com/v3/businesses/search?' + f"latitude={latitude}&longitude={longitude}" + f"&radius={radius}" + f"&categories={categories}"
    
    # Create dictionary for headers, add key with our API KEY
    headers = {"Accept": "application/json",
              "Authorization": f"Bearer {api_key}"}
    
    # Get the response, check status
    yelp_result = requests.get(yelp_url, headers=headers)
    
    return yelp_result.json()

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Start with the first 400 station coordinates from 'montreal_stations_coordinate1'

# Create an empty DataFrame to store the results from Yelp for bars with 1000m radius from Montreal bike stations
yelp_bars_df1 = pd.DataFrame()

for index, row in montreal_stations_coordinate1.iterrows():
    latitude = row['cb_latitude']
    longitude = row['cb_longitude']
    yelp_bar_data1 = get_yelp_business(latitude=latitude, longitude=longitude, radius=1000, api_key=YELP_KEY1, categories="bars")
    
    # Extract the 'businesses' key from the JSON response and create a DataFrame from it
    yelp_businesses1 = yelp_bar_data1.get('businesses', [])
    yelp_bar_details1 = pd.DataFrame(yelp_businesses1)

    # Add latitude and longitude to yelp_bar_details1 DataFrame
    yelp_bar_details1['yelp_latitude'] = latitude
    yelp_bar_details1['yelp_longitude'] = longitude

    # Append it to 'yelp_bars_df1'
    yelp_bars_df1 = pd.concat([yelp_bars_df1, yelp_bar_details1], ignore_index=True)

In [ ]:
yelp_bars_df1.head()

In [ ]:
# Save the dataframe 'yelp_bars_df1' as 'yelp_bars_df1.csv' to archive the first API call data for safety
yelp_bars_df1.to_csv('../data/yelp_bars_df1.csv')

In [ ]:
# Due to Yelp's limit of 500 requests per day, we can get creative with another API key for the rest of Montreal station coordinates
YELP_KEY2 = os.getenv('yelp_key2')

In [ ]:
# Continue with rest of station coordinates with 'montreal_stations_coordinate2' (398 rows)

# Create an empty DataFrame to store the results from Yelp for bars with 1000m radius from Montreal bike stations
yelp_bars_df2 = pd.DataFrame()

for index, row in montreal_stations_coordinate2.iterrows():
    latitude = row['cb_latitude']
    longitude = row['cb_longitude']
    yelp_bar_data2 = get_yelp_business(latitude=latitude, longitude=longitude, radius=1000, api_key=YELP_KEY2, categories="bars")
    
    # Extract the 'businesses' key from the JSON response and create a DataFrame from it
    yelp_businesses2 = yelp_bar_data2.get('businesses', [])
    yelp_bar_details2 = pd.DataFrame(yelp_businesses2)

    # Add latitude and longitude to yelp_bar_details1 DataFrame
    yelp_bar_details2['yelp_latitude'] = latitude
    yelp_bar_details2['yelp_longitude'] = longitude

    # Append it to 'yelp_bars_df2'
    yelp_bars_df2 = pd.concat([yelp_bars_df2, yelp_bar_details2], ignore_index=True)

In [ ]:
yelp_bars_df2.head()

In [ ]:
# Save the dataframe 'yelp_bars_df2' as 'yelp_bars_df2.csv' to archive the second API call data for safety
yelp_bars_df2.to_csv('../data/yelp_bars_df2.csv')

In [ ]:
# Check the shape of the two API results before merging into one dataframe
print(yelp_bars_df1.shape)
print(yelp_bars_df2.shape)

In [ ]:
# The columns of the two results match
yelp_bars_df1.columns == yelp_bars_df2.columns

In [ ]:
# Concatenate the two results into the 'merged_yelp_bars_df' dataframe
merged_yelp_bars_df = pd.concat([yelp_bars_df1, yelp_bars_df2], ignore_index=True)
merged_yelp_bars_df.info()

In [ ]:
# Convert into 'merged_yelp_bars_df.csv' file for archival
merged_yelp_bars_df.to_csv('../data/merged_yelp_bars_df.csv')

In [ ]:
# This master Yelp dataframe provides rating for bars whereas Foursquare doesn't
# Check if bar 'id' is duplicated, as they might be good for imputation
yelp_dup = merged_yelp_bars_df[merged_yelp_bars_df[['id']].duplicated() == True].sort_values('id')
yelp_dup

Those duplicated rows can't be eliminated yet. While the majority of columns look the same, we used coordinates of the bike stations in Montreal to look up bars nearby. We'll revisit this issue in part 3.

Put your parsed results into a DataFrame

In [ ]:
merged_yelp_bars_df.columns

In [ ]:
# Initialize lists to only select data from some meaningful columns: name, is_closed, review_count, rating, price, zip_code, distance
name = []
zip_code = []
category = []
review_count = []
rating = []
price = []

distance = []

# Extract data from 'result_df'
for index, record in merged_yelp_bars_df.iterrows():
    name.append(record['name'])
    zip_code.append(location['zip_code'])

    # Extract the category alias with "bar" from the categories alias
    categories = record['categories']
    for alias in categories:
        if 'bar' in category['alias']:
            category.append(row)
            break

    review_count.append(record['review_count'])
    rating.append(record['rating'])
    price.append(record['price'])
    location = record['location']

    distance.append(record['distance'])

# Create a DataFrame from the extracted data
yelp_bar_features_df = pd.DataFrame({
    'name': name,
    'review_count': review_count,
    'rating': rating,
    'price': price,
    'postcode': zip_code,
    'distance': distance
})

# Print the head of the DataFrame
yelp_bar_features_df.head(5)

In [ ]:
# Save the Yelp's Montreal bar result with desired columns to 'yelp_bar_features.csv'
yelp_bar_features_df.to_csv('../data/yelp_bar_features.csv')

# Comparing Results

**Which API provided you with more complete data? Provide an explanation.**

Comparing both the pre-processed and cleaned up data (mostly removing duplicated ids) responses from Foursquare and Yelp API, Yelp has a slightly better edge in complete data; regarding Montreal bar places within a kilometer away from the shared bike stations.

* **Foursquare**: 5907 entries with 10 columns initially received from API - 482 unique bar places returned after cleanup with more modest information about the place. But they have geographical indicators specific to the place (like geocodes, timezone) as well as connected places (chains, related places)
* **Yelp**: 5220 entries with 16 columns initially received from API - 482 unique bar places returned after cleanup with incredible indicators such as rating, review counts, prices and all.

Given the same amount of unique bars we get from the two API sources, we have the possibility to further explore whether these bars are the same. If true, there's no clear conclusion which API has a more completed data in terms of quantity.

**Get the top 10 restaurants or bars according to their rating**

In [ ]:
yelp_bar_features_df.sort_values(by='rating', ascending=False).head(10)